In [1]:
using Pkg; Pkg.activate("/home/dhairyagandhi96/modelzoo/model-zoo/script/../vision/mnist"); Pkg.instantiate(); Pkg.add(PackageSpec(name="CUDAdrv", rev = "master")); Pkg.add(PackageSpec(name="CUDAnative", rev = "master")); Pkg.add(PackageSpec(name="CuArrays", rev = "master")); Pkg.add(PackageSpec(name="Flux", rev = "master"))

using Flux, Flux.Data.MNIST
using Flux: @epochs, onehotbatch, mse, throttle
using Base.Iterators: partition
using Juno: @progress
using CuArrays

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/JuliaGPU/CUDAdrv.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/vision/mnist/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CUDAnative.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/vision/mnist/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CuArrays.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/vision/mnist/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/FluxML/Flux.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Pr

Encode MNIST images as compressed vectors that can later be decoded back into
images.

In [2]:
imgs = MNIST.images()

60000-element Array{Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt8,8}},2},1}:
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8

Partition into batches of size 1000

In [3]:
data = [float(hcat(vec.(imgs)...)) for imgs in partition(imgs, 1000)]
data = gpu.(data)

N = 32 # Size of the encoding

32

You can try to make the encoder/decoder network larger
Also, the output of encoder is a coding of the given input.
In this case, the input dimension is 28^2 and the output dimension of
encoder is 32. This implies that the coding is a compressed representation.
We can make lossy compression via this `encoder`.

In [4]:
encoder = Dense(28^2, N, leakyrelu) |> gpu
decoder = Dense(N, 28^2, leakyrelu) |> gpu

m = Chain(encoder, decoder)

loss(x) = mse(m(x), x)

evalcb = throttle(() -> @show(loss(data[1])), 5)
opt = ADAM(params(m))

@epochs 10 Flux.train!(loss, zip(data), opt, cb = evalcb)

┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Info: Epoch 1
└ @ Main.##749 /home/dhairyagandhi96/.julia/packages/Flux/eovHg/src/optimise/train.jl:103
┌ Warning: train!(loss, data, opt) is deprecated; use train!(loss, params, data, opt) instead
│   caller = ip:0x0
└ @ Core :-1
loss(data[1]) = 0.101888075f0 (tracked)
┌ Info: Epoch 2
└ @ Main.##749 /home/dhairyagandhi96/.julia/packages/Flux/eovHg/src/optimise/train.jl:103
┌ Info: Epoch 3
└ @ Main.##749 /home/dhairyagandhi96/.julia/packages/Flux/eovHg/src/optimise/train.jl:103
┌ Info: Epoch 4
└ @ Main.##749 /home/dhairyagandhi96/.julia/packages/Flux/eovHg/src/optimise/train.jl:103
┌ Info: Epoch 5
└ @ Main.##749 /home/dhairyagandhi96/.julia/packages/Flux/eovHg/src/optimise/train.jl:103
┌ Info: Epoch 6
└ @ Main.##749 /home/dhairyagandhi96/.julia/packages/Flux/eovHg/src/optimise/train.jl:103
┌ Info: Epoch 7
└ @ Main.##749 /home/dhairyagandhi96/.julia/packages/Flux/

Sample output

In [5]:
using Images

img(x::Vector) = Gray.(reshape(clamp.(x, 0, 1), 28, 28))

function sample()
  # 20 random digits
  before = [imgs[i] for i in rand(1:length(imgs), 20)]
  # Before and after images
  after = img.(map(x -> cpu(m)(float(vec(x))).data, before))
  # Stack them all together
  hcat(vcat.(before, after)...)
end

cd(@__DIR__)

save("sample.png", sample())